# RW-LM v4 spec functions

These functions will cover the functions need to update the layer spec and will be base on the code [develop for GFW migration to v3](https://github.com/Vizzuality/sci_team_data_bank/blob/master/Projects/GFW/layer_manager_migration/LM3_migration_functions.ipynb)  

we need to develop only migration methods for `gee, cartodb, featureservice, wms and leaflet` types

In [1]:
import json
import os
from copy import deepcopy
from time import sleep
from tqdm import tqdm
import subprocess

In [2]:
%run './pydantic_data_classes_rw.ipynb'

In [3]:
#install yarn globally
!npm install --global yarn

              ] | reify:yarn: timing reify:loadBundles Completed in 0msms
changed 1 package, and audited 2 packages in 568ms

found 0 vulnerabilities
npm notice 
npm notice New major version of npm available! 7.7.6 -> 8.3.0
npm notice Changelog: https://github.com/npm/cli/releases/tag/v8.3.0
npm notice Run npm install -g npm@8.3.0 to update!
npm notice 


In [4]:
## intall test suit for LM
#!git clone https://github.com/Vizzuality/jupyter-layer-test.git && cd jupyter-layer-test && yarn

In [5]:
## update test suit for LM
#!cd jupyter-layer-test && git pull && yarn && yarn build


In [6]:
class LayerValidation_error(BaseException):
    pass

In [7]:
def execute_test(layer: str)-> int:
    """
    Test that the layer works within layer manager
    """
    basePath = '/home/jovyan/work/notebooks-wri/layer-migration-v4'
    tests = f'{basePath}/jupyter-layer-test' # new algorithm , check=True
    #EXECUTES Tests
    os.chdir(tests)
    process = subprocess.run([f'yarn start \'{layer}\' '], shell=True)
    os.chdir(basePath)
    return process.returncode

In [8]:
def getCartoRasterLayerConfig(layer_config: dict) -> dict:
    """Creates new *carto* Layer Config based on previous one specific to those that are raster type"""
    ## Template config for Carto type layers
    new_lc = {
        "type": "raster",
        "lmMetadata":{"version": '4.0', "legacy-keys": None, "flag": 'Uses carto raster tiles.'},
        "source": {
                "type": "raster",
                "provider": {
                  "type": "carto",
                  "account": None,
                  "layers": [
                      {
                        "options": {
                          "sql": None,
                          "cartocss": None,
                          "cartocss_version": "2.3.0"
                        },
                        "type": "cartodb"
                      }
                    ]
                }
              },
        "render": {
            "layers": [] ### Stores vector styles
        }
    }
    
    # legacy keys
    new_lc["lmMetadata"]["legacy-keys"] = list(set(layer_config.keys()).difference(set(new_lc.keys())))

    # get account
    new_lc['source']['provider']['account'] = layer_config.get('account', 'wri-rw')

    # get layers, removing css (unused)
    new_lc['source']['provider']['layers'] = layer_config['body']['layers']
   
        
    return {**layer_config, **new_lc}

In [9]:
def getCartoLayerConfig(layer_config: dict) -> dict:
    """Creates new *carto* Layer Config based on previous one"""
    ## Template config for Carto type layers
    new_lc = {
        "type": "vector",
        "lmMetadata":{"version": '4.0', "legacy-keys": None},
        "source": {
                "type": "vector",
                "provider": {
                  "type": "carto",
                  "account": None,
                  "layers": [
                      {
                        "options": {
                          "sql": None
                        },
                        "type": "cartodb"
                      }
                    ]
                }
              },
        "render": {
                "layers": [] ### Stores vector styles
        }
    }
    
    # legacy keys
    new_lc["lmMetadata"]["legacy-keys"] = list(set(layer_config.keys()).difference(set(new_lc.keys())))

    # get account
    new_lc['source']['provider']['account'] = layer_config.get('account', 'wri-rw')

    # get layers, removing css (unused)
    new_lc['source']['provider']['layers'] = [{
                        "options": {
                            "sql": _l['options']['sql'],
                            "type": "cartodb"
                        }
                    } for _l in layer_config['body']['layers']]

    # add vector styles
    new_lc['render']["layers"] = layer_config['body']['vectorLayers']

    
        
    return {**layer_config, **new_lc}

In [10]:
def getTileLayerConfig(layer_config: dict) -> dict:
    """Creates new *tile_layer* Layer Config based on an existing one"""
    
    ## Template config for Carto type layers
    new_lc = {
      "type": "raster",
      "lmMetadata":{"version": '4.0', "legacy-keys": None},
      "source": {
        "type": "raster",
        "tiles": [],
        "minzoom": 3,
        "maxzoom": 12
      }
    }
    # legacy keys
    new_lc["lmMetadata"]["legacy-keys"] = list(set(layer_config.keys()).difference(set(new_lc.keys())))

    # get tile url
    if layer_config['body'].get('url', None):
        url = layer_config['body'].get('url', None)
        new_lc['source']['tiles'] = [url]
    elif layer_config.get('url', None):
        url = layer_config.get('url', None)
        new_lc['source']['tiles'] = [url]
    else:
        logging.debug(layer_config)
    
    # get min/maxzoom url
    minzoom = layer_config['body'].get('minzoom', None)
    maxzoom = layer_config['body'].get('maxzoom', None)
    if minzoom: new_lc['source']['minzoom'] = minzoom
    if maxzoom: new_lc['source']['maxzoom'] = maxzoom
        
    return {**layer_config, **new_lc}


In [11]:
# https://api.resourcewatch.org/v1/layer/b9d1b315-efc3-433a-82ad-22e841c34f5a/tile/gee/z/x/y

In [12]:
def getGeeLayerConfig(layer_config: dict) -> dict:
    """Creates new *tile_layer* Layer Config based on an existing one"""
    
    ## Template config for Carto type layers
    new_lc = {
      "type": "raster",
      "lmMetadata":{"version": '4.0', "legacy-keys": None, "flag": "gee legacy-keys are used for the internal tiler; dont delete"},
      "source": {
        "provider": {
          "type": 'gee',
          "options": {},
        },
        "type": "raster",
        "tiles":[],
        "minzoom": 3,
        "maxzoom": 12
      }
    }
    # legacy keys
    new_lc["lmMetadata"]["legacy-keys"] = list(set(layer_config.keys()).difference(set(new_lc.keys())))
        
    return {**layer_config, **new_lc}

In [13]:
def getFeatureServiceLayerConfig(layer_config: dict) -> dict:
    """Creates new *tile_layer* Layer Config based on an existing one"""
    
    ## Template config for FeatureServices arcgis layers
    new_lc = {
      "type": "raster",
      "lmMetadata":{"version": '4.0', "legacy-keys": None},
      "source": {
        "provider": {
          "type": 'feature-service',
          "options": {
            "tiler": None,
          },
        },
        "parse": False,
        "type": 'geojson',
        "minzoom": 3,
        "maxzoom": 12
      }
    }
    # legacy keys
    new_lc["lmMetadata"]["legacy-keys"] = list(set(layer_config.keys()).difference(set(new_lc.keys())))
    
    # get tile url
    url = None
    if layer_config['body'].get('url', None):
        url = layer_config['body'].get('url', None)
        
    elif layer_config.get('url', None):
        url = layer_config.get('url', None)
        
    else:
        logging.debug(layer_config)
    
    new_lc['source']['provider']['options']['tiler'] = url
        
    return {**layer_config, **new_lc}

In [14]:
def getWmsLayerConfig(layer_config: dict) -> dict:
    """Creates new *wms* Layer Config based on an existing one"""
    
    ## Template config for FeatureServices arcgis layers
    new_lc = {
      "type": "raster",
      "lmMetadata":{"version": '4.0', "legacy-keys": None},
      "source": {
        "provider": {
          "type": 'wms',
          "options": {
            #params for wms url composition
          },
        },
        "parse": False,
        "type": 'raster',
        "tiles": [
        ],
        "minzoom": 3,
        "maxzoom": 12
      }
    }
    # legacy keys
    new_lc["lmMetadata"]["legacy-keys"] = list(set(layer_config.keys()).difference(set(new_lc.keys())))
    
    # get tile url
    url = None
    if layer_config['body'].get('url', None):
        url = layer_config['body'].get('url', None)
        
    elif layer_config.get('url', None):
        url = layer_config.get('url', None)
        
    else:
        logging.debug(layer_config)
    
    new_lc['source']['tiles'] = [url]
        
    return {**layer_config, **new_lc}

In [15]:
def getMapboxLayerConfig(layer_config: dict) -> dict:
    """Creates new *mapbox* Layer Config based on previous one"""
    
    ## Template config for vectro tile layers
    new_lc = {
      "type": "vector",
      "lmMetadata":{"version": '4.0', "legacy-keys": None},
      "source": {
        "type": "vector",
        "url": ""
      },
    "render": {
            "layers": [] ### Stores vector styles
            }
    }
    
    # legacy keys
    new_lc["lmMetadata"]["legacy-keys"] = list(set(layer_config.keys()).difference(set(new_lc.keys())))
    
    # add vector tiles
    new_lc['source']["url"] = layer_config['body']['url']

    # add vector styles
    new_lc['render']["layers"] = layer_config['body']['vectorLayers']
        
    return {**layer_config, **new_lc}

In [16]:
def getGeojsonConfig(layer_config: dict) -> dict:
    """Creates new *geojson* Layer Config based on previous one"""
    
    new_lc = {
        "type": "vector",
        "lmMetadata":{"version": '4.0', "legacy-keys": None},
        "source": {
            "type": "geojson",
            "data": "", ### url
            "cluster": True,
            "clusterMaxZoom": 14,
            "clusterRadius": 45
              },
        "render": {
            "metadata": {
              "position": "top"
            },
            "layers": [] ### Stores vector styles
                }
    }
    
    # legacy keys
    new_lc["lmMetadata"]["legacy-keys"] = list(set(layer_config.keys()).difference(set(new_lc.keys())))

    # get clustering
    cluster = layer_config['body']['clusterConfig']
    new_lc['source'] = {**_lc['source'], **cluster}
    
    # get url
    url = layer_config['body']['url']
    if 'carto' in url: url += '&format=geojson'
    new_lc['source']['data'] = url

    # add vector styles
    vector_styles = layer_config['body']['vectorLayers']
    new_lc['render']["layers"] = vector_styles
        
    return {**layer_config, **new_lc}

In [17]:
def getVectorTileLayerConfig(layer_config: dict) -> dict:
    """Creates new *vector tile* Layer Config based on previous one"""
    
    ## Template config for vector tile layers
    _lc = {
      "type": "vector",
    "lmMetadata":{"version": '4.0', "legacy-keys": None},
      "source": {
        "type": "vector",
        "tiles": [],
        "minzoom": 3,
        "maxzoom": 12
      },
    "render": {
            "layers": [] ### Stores vector styles
            }
    }

    # copy tmp layer config
    new_lc = deepcopy(_lc)
    new_lc["lmMetadata"]["legacy-keys"] = list(set(layer_config.keys()).difference(set(new_lc.keys())))
    
    # add vector tiles
    tiles = layer_config['body']['tiles']
    new_lc['source']["tiles"] = tiles

    # add vector styles
    vector_styles = layer_config['body']['vectorLayers']
    new_lc['render']["layers"] = vector_styles

    # add params and timeline configs back
    if layer_config.get('params_config', None): new_lc['params_config'] = layer_config['params_config']
    if layer_config.get('timeline_config', None): new_lc['timeline_config'] = layer_config['timeline_config']
        
    return new_lc

In [19]:
## Todo remove prints with logger objects.
def createUpdatelayerSchemas(session, layer_id_list: list):
    """ Creates LM 4.0 jsons.
    
    Layer types GFW: ['carto', 'tile_layer', 'gee', 'vector_tile_layer', 'mapbox', wms]
    Layer types RW: ['gee', 'cartodb', 'featureservice', 'wms', 'leaflet']
    """
    layerProviders = {
        'carto': getCartoLayerConfig,
        'cartoRaster': getCartoRasterLayerConfig,
        'cartodb': getCartoLayerConfig,
        'gee': getGeeLayerConfig,
        'featureservice': getFeatureServiceLayerConfig,
        'tile_layer': getTileLayerConfig,
        'wms': getWmsLayerConfig,
        'leaflet': getTileLayerConfig,
        'vector_tile_layer': getVectorTileLayerConfig,
        'mapbox': getMapboxLayerConfig,
        'geojson': getGeojsonConfig
    }
    
    flagged = []
    failed = []
    success = []
    
    for layer_id in tqdm(layer_id_list):
        try:
            # get layer and create a new Layer object
            l: Layer = parse_obj_as(Layer, session.get(f'v1/layer/{layer_id}').json().get('data'))
            dataset_id: UUID = l.attributes._datasetId
            layer_type: str = l.attributes.provider
            
            if not l.attributes.layerConfig.get('body'):
                flagged.append(layer_id)
                continue
            # if no vector styles - we used it as a carto raster tile.
            if l.attributes.provider == 'cartodb'and l.attributes.layerConfig.get('body') and not l.attributes.layerConfig.get('body', {}).get('vectorLayers', None):
                layer_type = 'cartoRaster'
                
            new_lc = layerProviders[layer_type](l.attributes.layerConfig)

            l.attributes.layerConfig = new_lc

            ## Logic to validate layer
            #testResult = execute_test(f'{json.dumps({"id":0, **l.attributes.layerConfig})}')
            #if testResult != 0:
            #    raise LayerValidation_error(f'exit code: {testResult}')

            ## Logic to update layer
            session.patch(f'v1/dataset/{l.attributes._datasetId}/layer/{layer_id}', 
                          data = l.attributes.json(exclude_none=True))

            success.append(layer_id)

        except Exception as e:
            print(f'Layer - {l.attributes.name} failed!')
            print(l._url)
            print(f'{e}')
            failed.append(layer_id)
        except LayerValidation_error as e:
            print(f'Layer - {l.attributes.name} failed while validating schema!')
            print(f'{e}')
            failed.append(layer_id)

    print(f'Done! {len(failed)} / {len(success) + len(failed)}  failed, {len(flagged)} flagged.')
    
    return {'flagged': flagged, 'failed': failed, 'success': success}

In [1]:
## Todo remove prints with logger objects.
def createUpdateMapWidgetSchemas(session, widget_id_list: list):
    """ Updates widget config for map types with masks ussing LM 4.0 jsons.
    
    """
    flagged = []
    failed = []
    success = []
    
    for widget_id in tqdm(widget_id_list):
        try:
            # get widget and create a new Widget object
            w: Widget = parse_obj_as(Widget, session.get(f'v1/widget/{widget_id}').json().get('data'))
            dataset_id: UUID = w.attributes._datasetId
            
            if w.attributes.widgetConfig.get('type') != 'map':
                print('not a map widget, it cannot be updated')
                continue
                
            if not w.attributes.widgetConfig.get('paramsConfig',{}).get('mask'):
                print('map widget without mask; it cannot be updated')
                continue
            
            new_widgetConfig = w.attributes.widgetConfig
            
            new_mask = getCartoLayerConfig(w.attributes.widgetConfig.get('paramsConfig',{}).get('mask'))
            
            new_widgetConfig['paramsConfig']['mask'] = new_mask
            
            w.attributes.widgetConfig = new_widgetConfig

            ## Logic to update widget

            session.patch(f'v1/dataset/{dataset_id}/widget/{widget_id}', 
                          data = w.attributes.json(exclude_none=True))

            success.append(widget_id)

        except Exception as e:
            print(f'Widget - {w.attributes.name} failed!')
            print(w._url)
            print(f'{e}')
            failed.append(widget_id)
        except LayerValidation_error as e:
            print(f'Widget - {w.attributes.name} failed while validating schema!')
            print(f'{e}')
            failed.append(widget_id)

    print(f'Done! {len(failed)} / {len(success) + len(failed)}  failed, {len(flagged)} flagged.')
    
    return {'flagged': flagged, 'failed': failed, 'success': success}